In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
import time
url = 'https://www.vgchartz.com/games/games.php?name=&keyword=&console=&region=All&developer=&publisher=&goty_year=&genre=&boxart=Both&banner=Both&ownership=Both&showmultiplat=No&results=200&order=Sales&showtotalsales=0&showpublisher=0&showpublisher=1&showvgchartzscore=0&shownasales=0&showdeveloper=0&showcriticscore=0&showpalsales=0&showreleasedate=0&showreleasedate=1&showuserscore=0&showjapansales=0&showlastupdate=0&showothersales=0&showshipped=0&showshipped=1'

In [30]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url)
time.sleep(5)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
result = soup.find('th')
result = str(result)
result = result.split('(')[1]
result = result.split(')')[0]

counter = 2
list_1 = []
list_2 = []
list_3 = []
list_4 = []
egg_salad = ['<< 1 2 3 4 5 6 >>', '<< 1 2 3 4 5 6 7 >>', '<< 1 2 3 4 5 6 7 8 >>', '<< 1 2 3 4 5 6 7 8 9 >>', '<< 1 2 3 4 5 6 7 8 9 10 >>', '<< 1 2 3 4 5 6 7 8 9 10 11 >>', '<< 1 2 3 4 5 6 7 8 9 10 11 12 >>']

for item in egg_salad:
    try:
        html = browser.html
        time.sleep(5)
        print("Got the HTML!")
        df = pd.read_html(html)
        df = pd.DataFrame(df[6])
        try:
            field_1 = df[f'Results: ({result})']['A B C D E F G H I J K L M N O P Q R S T U V W X Y Z']['Game.1']
            field_2 = df[item]['A B C D E F G H I J K L M N O P Q R S T U V W X Y Z']['Publisher']
            field_3 = df[item]['A B C D E F G H I J K L M N O P Q R S T U V W X Y Z']['Total Shipped']
            field_4 = df[item]['A B C D E F G H I J K L M N O P Q R S T U V W X Y Z']['Release Date']
            print("Got all the fields!")
        except:
            continue

        col_1 = list(field_1)
        col_2 = list(field_2)
        col_3 = list(field_3)
        col_4 = list(field_4)
        for item in col_1:
            list_1.append(item)
        for item in col_2:
            list_2.append(item)
        for item in col_3:
            list_3.append(item)
        for item in col_4:
            list_4.append(item)
        browser.find_by_xpath(f'/html/body/div[3]/div/div[2]/table/tbody/tr/td/div/div[2]/table[1]/tbody/tr[1]/th[2]/span[{counter}]/a').click()
        counter += 1
        print("Success!")
        time.sleep(5)
    except:
        counter += 1
        continue

browser.quit()

new_df = pd.DataFrame(list(zip(list_1, list_2, list_3, list_4)), columns=['title', 'publisher', 'sales', 'released'])
new_df = new_df.dropna()
new_df

Got the HTML!
Got all the fields!
Success!
Got the HTML!
Got all the fields!
Success!
Got the HTML!
Got all the fields!
Success!
Got the HTML!
Got all the fields!
Hmmm. Error
Got the HTML!
Got all the fields!
Hmmm. Error


,title,publisher,sales,released
0,Pokemon,Nintendo,365.60m,28th Sep 98
1,Super Mario,Nintendo,356.59m,20th Jul 83
2,Grand Theft Auto,ASC Games,310.00m,27th Mar 98
3,Call of Duty,Activision,300.00m,29th Oct 03
4,FIFA,EA Sports,282.40m,15th Dec 93
...,...,...,...,...
991,Mario Party 9,Nintendo,3.11m,11th Mar 12
992,Parasite Eve,Square,3.10m,09th Sep 98
994,The Legend of Zelda: Majora's Mask 3D,Nintendo,3.09m,13th Feb 15
996,World Class Track Meet,Bandai,3.08m,01st Sep 87


In [36]:
rds_connection_string = "postgres:postgres@localhost:5432/game_db"
engine = create_engine(f'postgresql://{rds_connection_string}')
new_df.to_sql(name='games', con=engine, if_exists='append', index=False)